# Анализ тональности текстов на русском:

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/russian-slang/russian_slang.csv
/kaggle/input/rusentiment/rusentiment_random_posts.csv
/kaggle/input/rusentiment/rusentiment_test.csv
/kaggle/input/rusentiment/rusentiment_preselected_posts.csv
/kaggle/input/emoji-with-russian-description/emoji_dataset_russian.csv


In [2]:
import torch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


import re, string
import emoji
import nltk

from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

import torch

from sklearn.model_selection import train_test_split
import datasets
from datasets import Dataset , Sequence , Value , Features , ClassLabel , DatasetDict

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import word_tokenize

from tqdm import tqdm
from collections import defaultdict

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

#metrics
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

seed=42

sns.set_style("whitegrid")
sns.despine()
plt.style.use("seaborn-v0_8-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc("axes", labelweight="bold", labelsize="large", titleweight="bold", titlepad=10)


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<Figure size 640x480 with 0 Axes>

In [4]:
def conf_matrix(y, y_pred, title):
    fig, ax =plt.subplots(figsize=(5,5))
    labels=['Negative', 'Neutral', 'Positive']
    ax=sns.heatmap(confusion_matrix(y, y_pred), annot=True, cmap="Blues", fmt='g', cbar=False, annot_kws={"size":25})
    plt.title(title, fontsize=20)
    ax.xaxis.set_ticklabels(labels, fontsize=17)
    ax.yaxis.set_ticklabels(labels, fontsize=17)
    ax.set_ylabel('Test', fontsize=20)
    ax.set_xlabel('Predicted', fontsize=20)
    plt.show()

# Загрузка данных

In [5]:
from datasets import load_dataset

ds = load_dataset("MonoHime/ru_sentiment_dataset")
#ds_second = pd.read_csv("/kaggle/input/rusentiment/rusentiment_random_posts.csv")
#ds_test = pd.read_csv('/kaggle/input/rusentiment/rusentiment_test.csv')
df_emoji = pd.read_csv("/kaggle/input/emoji-with-russian-description/emoji_dataset_russian.csv")
df_slang = pd.read_csv("/kaggle/input/russian-slang/russian_slang.csv")

README.md:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/276M [00:00<?, ?B/s]

valid.csv:   0%|          | 0.00/32.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/189891 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21098 [00:00<?, ? examples/s]

In [6]:
df_slang

,acronym,expansion
0,Баклажан,"Недовольство, обида"
1,Базарить,Разговаривать
2,Болтать,"Пустые разговоры, трёп"
3,Бомба,"Очень круто, классно"
4,Влететь,Попасть в неприятности
...,...,...
95,Ёшкин кот,Восклицание удивления
96,Юзать,Использовать
97,Ясно как день,Понять всё на 100%
98,Ячейка,"Часть, элемент чего-то"


In [7]:
#ds_second

In [8]:
#ds_second = ds_second[~ds_second['label'].isin(['speech', 'skip'])]

In [9]:
#ds_test = ds_test[~ds_test['label'].isin(['speech', 'skip'])]

In [10]:
df_emoji

,emoji,description_ru
0,🥇,:медаль за 1 место:
1,🥈,:медаль за 2 место:
2,🥉,:медаль за 3 место:
3,🆎,Кнопка :AB (группа крови):
4,🏧,:Знак банкомата:
...,...,...
5037,🤪,:сумасшедшее лицо:
5038,🦓,:зебра:
5039,🤐,:лицо с застегнутым ртом:
5040,🧟,:зомби:


In [11]:
#ds_second

# Очистка

In [12]:
train_data = ds['train'].to_pandas()
val_data = ds['validation'].to_pandas()

In [13]:
train_data

,Unnamed: 0,text,sentiment
0,21098,".с.,и спросил его: о Посланник Аллаха!Ты пори...",1
1,21099,Роднее всех родных Попала я в ГКБ №8 еще в дек...,1
2,21100,Непорядочное отношение к своим работникам Рабо...,2
3,21101,"). Отсутствуют нормативы, Госты и прочее, что ...",1
4,21102,У меня машина в руках 5 лет и это п...,1
...,...,...,...
189886,210984,"Мой юбилей я отмечал в ресторане "" Астория "" ....",2
189887,210985,"Отлично встретили, разместили в роскошном номе...",1
189888,210986,Была в Васаби на ст. метро Сенная . Во первых...,0
189889,210987,Ребята не стоит смотреть этот фильм. Вы молоды...,0


In [14]:
ds = train_data.drop(columns=['Unnamed: 0'])


ds = ds.rename(columns={'sentiment': 'label'})

In [15]:
ds_val = val_data.drop(columns=['Unnamed: 0'])


ds_val = ds_val.rename(columns={'sentiment': 'label'})

In [16]:
#ds_second['label'] = ds_second['label'].map({'negative':2,'neutral':0,'positive':1})
#ds_test['label'] = ds_test['label'].map({'negative':2,'neutral':0,'positive':1})

In [17]:
ds['label'].value_counts()

label
1    90766
2    49798
0    49327
Name: count, dtype: int64

In [18]:
#ds = pd.concat([ds, ds_second, ds_test, ds_val], ignore_index=True)

In [19]:
ds = pd.concat([ds, ds_val], ignore_index=True)

In [20]:
ds['label'].value_counts()

label
1    100792
2     55310
0     54887
Name: count, dtype: int64

In [21]:
    #def limit_classes_to_n(df, class_column, n=1000):
    
   #     unique_classes = df[class_column].unique()
        
    #    limited_df = pd.DataFrame(columns=df.columns)
        
   #     for cls in unique_classes:
      #      class_rows = df.query(f"{class_column} == @cls")
            
      #      limited_class_rows = class_rows.head(n)
            
        #    limited_df = pd.concat([limited_df, limited_class_rows])
        
     #   return limited_df
    
   # full_df = limit_classes_to_n(ds, 'label',  50000)
   # full_df['label'].value_counts()

In [22]:
df, df_test = train_test_split(ds, test_size=0.2, stratify=ds['label'])

In [23]:
df_train, df_val = train_test_split(df, test_size=0.1, random_state=RANDOM_SEED, stratify=df['label'])
print(df_train.shape , df_val.shape, df_test.shape)

(151911, 2) (16880, 2) (42198, 2)


In [24]:
def createDataset(df, textCol, labelCol):
  dataset_dict = {
    'text' : df[textCol],
    'labels' : df[labelCol],
  }
  sent_tags = ClassLabel(num_classes=3 , names=[ 'Neutral','Positive','Negative'])

  return Dataset.from_dict(
    mapping = dataset_dict,
    features = Features({'text' : Value(dtype='string') , 'labels' :sent_tags})
  )

In [25]:
dataset_train = createDataset(df_train,"text","label")
dataset_val = createDataset(df_val,"text","label")
dataset_test = createDataset(df_test,"text","label")

dataset_sentAnalysis = DatasetDict()
dataset_sentAnalysis["train"] = dataset_train
dataset_sentAnalysis["val"] = dataset_val
dataset_sentAnalysis["test"] = dataset_test

dataset_sentAnalysis

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 151911
    })
    val: Dataset({
        features: ['text', 'labels'],
        num_rows: 16880
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 42198
    })
})

**Then we define custom functions to clean the text of the tweets.**

In [26]:
emoji_dict = dict(zip(df_emoji['emoji'], df_emoji['description_ru']))
slang_dict = dict(zip(df_slang['acronym'], df_slang['expansion']))


def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def convert_to_lower(text):
    return text.lower()

def replace_emoji_with_words(text):
    for emj, desc in emoji_dict.items():
        text = text.replace(emj, f" {desc} ")
    return text

def replace_slang_with_full_form(text):
    words = text.split()
    new_words = [slang_dict.get(word.lower(), word) for word in words]
    return ' '.join(new_words)

def remove_extra_white_spaces(text):
    return re.sub(r'\s+', ' ', text).strip()


def preprocessText(text):
    text = convert_to_lower(text)
    text = replace_emoji_with_words(text)
    text = replace_slang_with_full_form(text)
    text = remove_punctuation(text)
    text = remove_extra_white_spaces(text)
    return text

def preprocessBatch(batch):
    
    processed_texts = []
    
    for text in batch["text"]:
        
        text = convert_to_lower(text)
        text = replace_emoji_with_words(text)
        text = replace_slang_with_full_form(text)
        text = remove_punctuation(text)
        text = remove_extra_white_spaces(text)  
       
        processed_texts.append(text)
    
    batch["text"] = processed_texts
    return batch

In [27]:
dataset_sentAnalysis_preprocessed = dataset_sentAnalysis.map(preprocessBatch, batched=True, batch_size=32)

Map:   0%|          | 0/151911 [00:00<?, ? examples/s]

Map:   0%|          | 0/16880 [00:00<?, ? examples/s]

Map:   0%|          | 0/42198 [00:00<?, ? examples/s]

In [28]:
text = "This game is dope! 🎮 Lmk when you're online!"

processed_text = preprocessText(text)
print("Оригинал:", text)
print("После обработки:", processed_text)

Оригинал: This game is dope! 🎮 Lmk when you're online!
После обработки: this game is dope видеоигра lmk when youre online


In [29]:
model_name = 'seara/rubert-tiny2-russian-sentiment'

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/368 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [30]:
max_len = 512
def tokenize(batch):
    tokens = tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=max_len
    )

    return tokens
    

In [31]:
sample_text = "What is  going on @resturant.:( It makes   Me Feel Upset.😞"
tokens = tokenizer.tokenize(sample_text)
print(len(tokens), tokens)
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(len(token_ids), token_ids)

token_dictionary = tokenizer(sample_text)
print(token_dictionary)
print(len(token_dictionary.input_ids))

tokens = tokenizer.convert_ids_to_tokens(token_dictionary['input_ids'])
print(tokens)

token_encode = tokenizer.encode(sample_text)
print(token_encode)

20 ['What', 'is', 'going', 'on', '@', 'rest', '##uran', '##t', '.', ':', '(', 'It', 'makes', 'Me', 'Fe', '##el', 'Up', '##set', '.', '[UNK]']
20 [2106, 550, 5342, 561, 36, 4529, 9166, 549, 18, 30, 12, 771, 5979, 1569, 5813, 904, 2732, 3169, 18, 1]
{'input_ids': [2, 2106, 550, 5342, 561, 36, 4529, 9166, 549, 18, 30, 12, 771, 5979, 1569, 5813, 904, 2732, 3169, 18, 1, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
22
['[CLS]', 'What', 'is', 'going', 'on', '@', 'rest', '##uran', '##t', '.', ':', '(', 'It', 'makes', 'Me', 'Fe', '##el', 'Up', '##set', '.', '[UNK]', '[SEP]']
[2, 2106, 550, 5342, 561, 36, 4529, 9166, 549, 18, 30, 12, 771, 5979, 1569, 5813, 904, 2732, 3169, 18, 1, 3]


In [32]:
print(tokenizer)
print(tokenizer.sep_token, tokenizer.sep_token_id)
print(tokenizer.cls_token, tokenizer.cls_token_id)
print(tokenizer.pad_token, tokenizer.pad_token_id)
print(tokenizer.unk_token, tokenizer.unk_token_id)

BertTokenizerFast(name_or_path='seara/rubert-tiny2-russian-sentiment', vocab_size=83828, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
[SEP] 3
[CLS] 2
[PAD] 0
[UNK] 1


In [33]:
encoding = tokenizer.encode_plus(
    sample_text,
    max_length=max_len,
    truncation=True,
    add_special_tokens=True,
    padding='max_length',
    return_attention_mask=True,
    return_token_type_ids=False,
    return_tensors="pt"
)
encoding

{'input_ids': tensor([[   2, 2106,  550, 5342,  561,   36, 4529, 9166,  549,   18,   30,   12,
          771, 5979, 1569, 5813,  904, 2732, 3169,   18,    1,    3,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,

In [34]:
dataset_sentAnalysis_encoded = dataset_sentAnalysis_preprocessed.map(tokenize, batched=True, batch_size=32)

Map:   0%|          | 0/151911 [00:00<?, ? examples/s]

Map:   0%|          | 0/16880 [00:00<?, ? examples/s]

Map:   0%|          | 0/42198 [00:00<?, ? examples/s]

In [35]:
dataset_sentAnalysis_encoded

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 151911
    })
    val: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16880
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 42198
    })
})

# Даталоадер

In [36]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_name)
print(config.max_position_embeddings)

config.json:   0%|          | 0.00/911 [00:00<?, ?B/s]

2048


In [37]:
from torch.utils import data
class SentimentDataset(data.Dataset):

  def __init__(self, text, targets, tokenizer, max_len=512, transform=None):
    self.text = text
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.transform = transform

  def __len__(self):
    return len(self.text)

  def __getitem__(self,itemInd):
    text =  str(self.text[itemInd])


    text = preprocessText(text)

    if self.transform:
        text = self.transform(self.text)

    encoding = tokenizer.encode_plus(
        text,
        max_length=self.max_len,
        add_special_tokens=True,
        truncation=True,
        padding='max_length',
        return_attention_mask=True,
        return_token_type_ids=False,
        return_tensors="pt"
    )


    return {
        'text': text,
        'input_ids': encoding["input_ids"].flatten(),
        'attention_mask': encoding["attention_mask"].flatten(),
        'targets': torch.tensor(self.targets[itemInd], dtype=torch.long)
    }

def create_dataloader(df, tokenizer, max_len, batch_size):
    ds = SentimetDataset(
        text = df["text"].to_numpy(),
        targets = df["label"].to_numpy(),
        tokenizer = tokenizer,
        max_len = max_len
    )

    return data.DataLoader (
        ds,
        batch_size = batch_size,
        num_workers = 0
    )

In [38]:
batch_size = 32

train_data_loader = create_dataloader(df_train, tokenizer, max_len, batch_size)
val_data_loader = create_dataloader(df_val, tokenizer, max_len, batch_size)
test_data_loader = create_dataloader(df_test, tokenizer, max_len, batch_size)
it = iter(train_data_loader)
data_batch = next(it)
data_batch.keys()

print(data_batch["input_ids"].shape, data_batch["attention_mask"].shape, data_batch["targets"].shape)
data_batch["input_ids"][0]

torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32])


tensor([    2,  3570, 26629, 46497,   548, 29670, 15602,   326,  5151, 26629,
        40810, 22933,  3981,   320, 67718,  1150, 36618,   626,   314, 49339,
          320, 52997,   626, 46716,   791, 15602, 33048,   733, 10638,  2085,
          644,  1363, 30143, 13786, 51297, 32952,   548,  5509, 11491,   320,
        29642, 18610, 43965,  4963, 14151,   320, 36177, 15602,  2389, 39155,
        83340, 41265, 24681,  5557, 25599, 64063, 13796, 46087, 47460, 51048,
         3272,  8621, 25913,  6796, 14151, 30139, 18793, 10030, 81204,   320,
        63118,   871, 32668, 36069, 49079, 18589,  2179, 79348,   761, 82554,
        23917,  2841, 48591,   329, 73059,  2371, 11521, 33166,  1150, 78338,
        32668,   314, 24890, 81204, 61097, 19091, 37394, 55719, 36782,   769,
        36019,   548, 53638, 13009,   769, 29727, 32412, 31698,  7081, 14151,
         2399, 32986,   322,  6032,   644,  1619,   871, 29642,  4674, 32925,
          548, 43412,   314,    28, 33643,  1218,  3687, 58785, 

# Модель


In [39]:
from transformers import BertConfig
from transformers.models.bert.modeling_bert import BertModel, BertPreTrainedModel
from transformers.modeling_outputs import SequenceClassifierOutput
import torch.nn as nn

class BertForClassification(BertPreTrainedModel):

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config

        self.bert = BertModel(config)

        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        self.post_init()


    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None,
                labels=None, **kwargs):
        
        outputs = self.bert(input_ids, attention_mask=attention_mask,
                               token_type_ids=token_type_ids, **kwargs)

        sequence_output = self.dropout(outputs[1])
        logits = self.classifier(sequence_output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [40]:
from transformers import AutoConfig

id2label = {
    0:  'Neutral',
    1: 'Positive',
    2: 'Negative'
}

label2id = { v:k for (k,v) in id2label.items()}

bert_config = AutoConfig.from_pretrained(model_name,
                                         num_labels=3,
                                         id2label=id2label, label2id=label2id)

In [41]:
bert_model = (BertForClassification
              .from_pretrained(model_name, config=bert_config).to(device))

model.safetensors:   0%|          | 0.00/117M [00:00<?, ?B/s]

# Обучение

In [42]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [43]:
from torch.optim import AdamW
from transformers import get_scheduler

lr=2e-5
optimizer = AdamW(bert_model.parameters(), lr=lr)

num_epochs = 4

logging_steps = len(dataset_sentAnalysis_encoded["train"]) // batch_size
num_training_steps = num_epochs * logging_steps
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [44]:
loss_fct = nn.CrossEntropyLoss().to(device)

In [45]:
from tqdm import tqdm

def eval_op(model, data_loader, loss_fn, n_examples):
    model.eval()
    losses = []
    correct_predictions = 0

    with tqdm(total=len(data_loader), desc="Validation", unit="batch") as pbar:
        with torch.no_grad():
            for d in data_loader:
                input_ids = d["input_ids"].to(device)
                attention_mask = d["attention_mask"].to(device)
                targets = d["targets"].to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                preds = torch.max(outputs.logits, dim=1)

                loss = loss_fn(outputs.logits, targets)
                correct_predictions += torch.sum(preds.indices == targets)
                losses.append(loss.item())

                pbar.set_postfix({"loss": f"{np.mean(losses):.4f}"})
                pbar.update(1)

    return correct_predictions.double() / n_examples, np.mean(losses)


In [46]:
def train_epoch(
    model,
    data_loader,
    loss_fn,
    optimizer,
    n_examples,
    scheduler=None
):
    model.train()
    losses = []
    correct_predictions = 0

    with tqdm(total=len(data_loader), desc="Training", unit="batch") as pbar:
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)
            
            optimizer.zero_grad()
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.max(outputs.logits, dim=1)

            loss = loss_fn(outputs.logits, targets)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            correct_predictions += torch.sum(preds.indices == targets)
            losses.append(loss.item())

            pbar.set_postfix({"loss": f"{np.mean(losses):.4f}"})
            pbar.update(1)

            if scheduler:
                scheduler.step()

    return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
%%time
history = defaultdict(list)
best_accuracy = 0

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    print("-" * 10)

    train_acc, train_loss = train_epoch(
        bert_model,
        train_data_loader,
        loss_fct,
        optimizer,
        len(df_train),
        scheduler=lr_scheduler
    )
    print(f"Train loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}")

    val_acc, val_loss = eval_op(
        bert_model,
        val_data_loader,
        loss_fct,
        len(df_val)
    )
    print(f"Val loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}")

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)

 

Epoch 1/4
----------


Training:   1%|          | 24/4748 [04:00<12:14:44,  9.33s/batch, loss=1.0027]

# **Валидируем**

In [ ]:
test_acc, test_loss = eval_op(bert_model, test_data_loader, loss_fct, len(df_test))
test_acc, test_loss

In [ ]:
def get_text(model, data_loader):

  model = model.eval()
  sentiment_texts = []
  predictions = []
  prediction_probs = []
  real_values = []

  with torch.no_grad():
    for d in data_loader:
      texts = d["text"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)
      
      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _,preds = torch.max(outputs.logits, dim=1)
      
      sentiment_texts.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(outputs.logits)
      real_values.extend(targets)

  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()

  return sentimet_texts, predictions, prediction_probs, real_values

In [ ]:
y_sentiment_texts, y_pred, y_probs, y_test = get_text(bert_model, test_data_loader)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

class_names = ['Neutral','Positive','Negative']
print(classification_report(y_test, y_pred, target_names=class_names))

In [ ]:
def show_confusion_matrix(confusion_matrix):
  hmap = sns.heatmap(confusion_matrix, annot=True, fmt=".2f", cmap="Blues")
  hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
  hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
  plt.ylabel('True sentiment')
  plt.xlabel('Predicted sentiment')

cm = confusion_matrix(y_test, y_pred, normalize="true")
df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
show_confusion_matrix(df_cm)

In [ ]:
sample_text = "I buy a new house. This is great for me." 
sample_text = preprocessText(sample_text)
encoding = tokenizer.encode_plus(
    sample_text,
    max_length=max_len,
    add_special_tokens=True,
    pad_to_max_length=True,
    truncation=True,
    return_attention_mask=True,
    return_token_type_ids=False,
    return_tensors="pt"
)

input_ids = encoding["input_ids"].to(device)
attention_mask = encoding["attention_mask"].to(device)

output = bert_model(input_ids, attention_mask)
_,preds = torch.max(output.logits, dim=1)
print(preds)
print(class_names[preds])


pd.DataFrame({
    "Text": sample_text,
    "Sentiment": class_names[preds]
},index=[0]).T

In [ ]:
torch.save({
            'epoch': epoch,
            'model_state_dict': bert_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss_fct
            }, '/kaggle/working/bert_model_weights.pth')

In [ ]:
torch.save(bert_model, '/kaggle/working/bert_full_model.pth')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device) 

In [ ]:
text = "I'm so hyped about this event! 🔥"


inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

In [ ]:
print("Model device:", next(bert_model.parameters()).device)


for k, v in inputs.items():
    print(f"{k} device: {v.device}")

In [ ]:
from IPython.display import FileLink
FileLink(r'bert_model_weights.pth')